<a href="https://colab.research.google.com/github/roberthouston14/githubtest/blob/main/IMDB_text_classification_Final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1tayKZyBpQX0nswLEjX6f7smUwvHii6D9?usp=sharing)

# Text classification - IMDB Dataset

In [ ]:
!pip install transformers[sentencepiece] datasets

## Datasets library

In [ ]:
from datasets import list_datasets
list_datasets()

In [4]:

!pip install torch
!pip install tokenizers
import torch
import transformers
from transformers import RobertaTokenizer, RobertaTokenizerFast


from transformers import RobertaTokenizer, RobertaModel
import torch

tokenizer = RobertaTokenizer.from_pretrained("ehsanaghaei/SecureBERT")
model = RobertaModel.from_pretrained("ehsanaghaei/SecureBERT")


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Some weights of the model checkpoint at ehsanaghaei/SecureBERT were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.layer_norm.bias', 'lm_head.dense.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaModel were not initialized from the model checkpoint at ehsanaghaei/SecureBERT and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
from datasets import load_dataset

imdb = load_dataset("imdb")
imdb

  0%|          | 0/3 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

- Similar to a python dictionary, where each key corresponds to a different split

In [6]:
imdb['train'][0]

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

In [7]:
imdb['test'][:3]

{'text': ['I love sci-fi and am willing to put up with a lot. Sci-fi movies/TV are usually underfunded, under-appreciated and misunderstood. I tried to like this, I really did, but it is to good TV sci-fi as Babylon 5 is to Star Trek (the original). Silly prosthetics, cheap cardboard sets, stilted dialogues, CG that doesn\'t match the background, and painfully one-dimensional characters cannot be overcome with a \'sci-fi\' setting. (I\'m sure there are those of you out there who think Babylon 5 is good sci-fi TV. It\'s not. It\'s clichéd and uninspiring.) While US viewers might like emotion and character development, sci-fi is a genre that does not take itself seriously (cf. Star Trek). It may treat important issues, yet not as a serious philosophy. It\'s really difficult to care about the characters here as they are not simply foolish, just missing a spark of life. Their actions and reactions are wooden and predictable, often painful to watch. The makers of Earth KNOW it\'s rubbish as

In [8]:
imdb['train'] = imdb['train'].shuffle(seed=1).select(range(2000))
imdb['train']

Dataset({
    features: ['text', 'label'],
    num_rows: 2000
})

In [9]:
imdb_train_validation = imdb['train'].train_test_split(train_size=0.8)
imdb_train_validation

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 1600
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 400
    })
})

In [10]:
imdb_train_validation['test']

Dataset({
    features: ['text', 'label'],
    num_rows: 400
})

In [11]:
imdb_train_validation['validation'] = imdb_train_validation.pop('test')
imdb_train_validation

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 1600
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 400
    })
})

In [12]:
imdb.update(imdb_train_validation)
imdb

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 1600
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 400
    })
})

In [13]:
imdb['test'] = imdb['test'].shuffle(seed=1).select(range(400))
imdb['test']

Dataset({
    features: ['text', 'label'],
    num_rows: 400
})

In [14]:
imdb['unsupervised'][:3]

{'text': ['This is just a precious little diamond. The play, the script are excellent. I cant compare this movie with anything else, maybe except the movie "Leon" wonderfully played by Jean Reno and Natalie Portman. But... What can I say about this one? This is the best movie Anne Parillaud has ever played in (See please "Frankie Starlight", she\'s speaking English there) to see what I mean. The story of young punk girl Nikita, taken into the depraved world of the secret government forces has been exceptionally over used by Americans. Never mind the "Point of no return" and especially the "La femme Nikita" TV series. They cannot compare the original believe me! Trash these videos. Buy this one, do not rent it, BUY it. BTW beware of the subtitles of the LA company which "translate" the US release. What a disgrace! If you cant understand French, get a dubbed version. But you\'ll regret later :)',
  'When I say this is my favourite film of all time, that comment is not to be taken lightly

In [15]:
imdb.pop('unsupervised')
imdb

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 1600
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 400
    })
    validation: Dataset({
        features: ['text', 'label'],
        num_rows: 400
    })
})

## Overview of IMDB Dataset

In [16]:
import pandas as pd
import matplotlib.pyplot as plt
pd.set_option('max_colwidth', 250)

In [17]:
imdb.set_format('pandas')
df = imdb['train'][:]
df.sample(frac=1 ,random_state=1).head(10)

,text,label
75,"This should be my kind of movie. Even if it sucked, it still should have been right up my alley; hell, I like ""Congo,"" and ""Allan Quatermaine"" movies. I have a soft spot in my heart for silly alien/demon/adventure movies. Let's go over why I deci...",0
1284,I'd been following this films progress for quite some time so perhaps expected a little too much. I consider both Gillian Anderson and Danny Dyer to be good at what they do and was interested to see what Dan Reed could come up with but unfortunat...,0
408,"I picked this title up from a friend who had it sitting in his exhaustive DVD/Video/Laserdisc collection, so luckily I didn't personally have to pay for it. I had an inkling that it would be a bad film, but I KNOW what a truly bad film is after w...",0
1282,"I can say nothing more about this movie than: Man, this SUCKS!!!!! If you really hate yourself and want to do some severe damage to your brain, watch this movie. It's the best cure in the world for taking away happiness. When I started watching t...",0
1447,"When I first heard about the title, I thought of 'The Simpsons', just like so many other reviewers, but when I saw the cast, I was completely stunned, that so many great character-actors would actually be in this! First of all, we have Christophe...",1
1144,"Who would've imagined -- Hal Hartley creates a filmic corollary to Syriana while retaining his signature idiosyncratic style. The fusion is highly entertaining.<br /><br />Having not seen a Hal Hartley film for about a decade, I approached this o...",1
1381,"I saw this film in a London cinema in 1975 and have not seen it since. I found it hilarious. I loved it's originality. It's rare that someone MAKES a movie like this - and it's sad too.<br /><br />What I mean is, I once read a book called ""The Bl...",1
181,"This series it's ""something different"". Sometimes European series are less accurate than the USA ones, but this time authors have hit the right target creating a mix that works in a smoothly way. Edel & Starck is great, it has all: great plot, sm...",1
1183,Pakeezah has a very interesting history (which is well documented in the 'Trivia' section) about how it came to be. It seems as if destiny conspired to test Kamal Amrohi (the director) while at the same time secretly desiring to see him complete ...,1
1103,"Wow. Who ever said that Edward D. Wood Jr. never influenced anybody? This steaming pile of donkey excrement is a perfect case in point; it makes ""The Violent Years"" look like ""Casablanca""! ""Santa Claus"" also makes Keith Richards' worst flashbacks...",0


In [18]:
df.loc[0, 'text']

'I have been searching for the right words to describe this film. At first I was inclined to simply skip to more significant matters, as the film does not rise to a level deserving comment.<br /><br />Yet, I stopped, puzzled as to how to describe such stuff and somewhat intrigued by the challenge presented by the question: "What can one say about such a film?" Rubbish? No, its not rubbish, rubbish can be recycled into something useful. Greenhouse gas? No, its not greenhouse gas, greenhouse gases help plants grow large and healthy.<br /><br />Finally, I struck on "Not even bad," a rework of the phrase "Not even wrong" used sometimes in theoretical physics to describe a theory that is hopelessly flawed and not even worthy of correction. That\'s it then, Georgia Rule: Not even bad.'

In [19]:
df['text'] = df.text.str.replace('<br />', '')
df.loc[0, 'text']

'I have been searching for the right words to describe this film. At first I was inclined to simply skip to more significant matters, as the film does not rise to a level deserving comment.Yet, I stopped, puzzled as to how to describe such stuff and somewhat intrigued by the challenge presented by the question: "What can one say about such a film?" Rubbish? No, its not rubbish, rubbish can be recycled into something useful. Greenhouse gas? No, its not greenhouse gas, greenhouse gases help plants grow large and healthy.Finally, I struck on "Not even bad," a rework of the phrase "Not even wrong" used sometimes in theoretical physics to describe a theory that is hopelessly flawed and not even worthy of correction. That\'s it then, Georgia Rule: Not even bad.'

In [21]:
df.label.value_counts()

0    818
1    782
Name: label, dtype: int64

In [ ]:
df["Words per review"] = df["text"].str.split().apply(len)
df.boxplot("Words per review", by="label", grid=False, showfliers=False,
           color="black")
plt.suptitle("")
plt.xlabel("")
plt.show()

In [ ]:
# 0 is negative
# 1 is positive
df[df.text.str.len() < 200]

,text,label,Words per review
137,One of my favorite scenes is at the beginning when guests on a private yacht decide to take an impromptu swim - in their underwear! Rather risqué for 1931!,1,29
685,"Before Dogma 95: when Lars used movies as art, not just a story. A beautiful painting about love and death. This is one of my favorite movies of all time. The color... The music... Just perfect.",1,36
756,Allison Dean's performance is what stands out in my mind watching this film. She balances out the melancholy tone of the film with an iridescent energy. I would like to see more of her.,1,34
761,"""Foxes"" is a great film. The four young actresses Jodie Foster, Cherie Currie, Marilyn Kagan and Kandice Stroh are wonderful. The song ""On the radio"" by Donna Summer is lovely. A great film. *****",1,34
1183,A movie best summed up by the scene where a victim simulates disembowelment by pulling some poor animal's intestines out from under her T-shirt. Too terrible for words.,0,28
1224,"My favorite part of this film was the old man's attempt to cure his neighbor's ills by putting the strong medicine in his bath. There is more than a sense of family, there is a sense of community.",1,38


In [23]:
imdb.reset_format()

## Tokenizer

In [62]:
from transformers import AutoTokenizer

#checkpoint = "distilbert-base-cased"
#checkpoint = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
# tokenizer = RobertaTokenizerFast.from_pretrained("ehsanaghaei/SecureBERT")
# checkpoint = transformers.RobertaForMaskedLM.from_pretrained("ehsanaghaei/SecureBERT")

def tokenize_function(batch):
    return tokenizer(batch["text"], padding=True, truncation=True)

imdb_encoded = imdb.map(tokenize_function, batched=True, batch_size=None) #.map method allows local endpoint RAM to be used to store elements of the dataset so 
imdb_encoded

  0%|          | 0/1 [00:00<?, ?ba/s]

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 1600
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 400
    })
    validation: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 400
    })
})

In [63]:
print(imdb_encoded['train'][0])

{'text': 'I have been searching for the right words to describe this film. At first I was inclined to simply skip to more significant matters, as the film does not rise to a level deserving comment.<br /><br />Yet, I stopped, puzzled as to how to describe such stuff and somewhat intrigued by the challenge presented by the question: "What can one say about such a film?" Rubbish? No, its not rubbish, rubbish can be recycled into something useful. Greenhouse gas? No, its not greenhouse gas, greenhouse gases help plants grow large and healthy.<br /><br />Finally, I struck on "Not even bad," a rework of the phrase "Not even wrong" used sometimes in theoretical physics to describe a theory that is hopelessly flawed and not even worthy of correction. That\'s it then, Georgia Rule: Not even bad.', 'label': 0, 'input_ids': [101, 146, 1138, 1151, 6205, 1111, 1103, 1268, 1734, 1106, 5594, 1142, 1273, 119, 1335, 1148, 146, 1108, 13427, 1106, 2566, 19476, 1106, 1167, 2418, 5218, 117, 1112, 1103, 12

## Tiny IMDB

In [64]:
import transformers
import re

[x for x in dir(transformers) if re.search(r'^AutoModel', x)]

['AutoModel',
 'AutoModelForAudioClassification',
 'AutoModelForAudioFrameClassification',
 'AutoModelForAudioXVector',
 'AutoModelForCTC',
 'AutoModelForCausalLM',
 'AutoModelForDepthEstimation',
 'AutoModelForDocumentQuestionAnswering',
 'AutoModelForImageClassification',
 'AutoModelForImageSegmentation',
 'AutoModelForInstanceSegmentation',
 'AutoModelForMaskedImageModeling',
 'AutoModelForMaskedLM',
 'AutoModelForMultipleChoice',
 'AutoModelForNextSentencePrediction',
 'AutoModelForObjectDetection',
 'AutoModelForPreTraining',
 'AutoModelForQuestionAnswering',
 'AutoModelForSemanticSegmentation',
 'AutoModelForSeq2SeqLM',
 'AutoModelForSequenceClassification',
 'AutoModelForSpeechSeq2Seq',
 'AutoModelForTableQuestionAnswering',
 'AutoModelForTokenClassification',
 'AutoModelForUniversalSegmentation',
 'AutoModelForVideoClassification',
 'AutoModelForVision2Seq',
 'AutoModelForVisualQuestionAnswering',
 'AutoModelForZeroShotObjectDetection',
 'AutoModelWithLMHead']

In [65]:
import torch
from transformers import AutoModelForSequenceClassification

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
num_labels = 2
model = (AutoModelForSequenceClassification
         .from_pretrained(checkpoint, num_labels=num_labels)
         .to(device))

In [ ]:
from datasets import DatasetDict

tiny_imdb = DatasetDict()
tiny_imdb['train'] = imdb['train'].shuffle(seed=1).select(range(50))
tiny_imdb['validation'] = imdb['validation'].shuffle(seed=1).select(range(10))
tiny_imdb['test'] = imdb['test'].shuffle(seed=1).select(range(10))

tiny_imdb_encoded = tiny_imdb.map(tokenize_function, batched=True, batch_size=None)
tiny_imdb_encoded

In [67]:
###<<<LOOK LOOK LOOK>>>
from transformers import Trainer, TrainingArguments

batch_size = 16
logging_steps = len(tiny_imdb_encoded["train"]) // batch_size
model_name = f"{checkpoint}-finetuned-tiny-imdb"
training_args = TrainingArguments(output_dir=model_name,
                                  num_train_epochs=3,
                                  learning_rate=2e-5,
                                  per_device_train_batch_size=batch_size,
                                  per_device_eval_batch_size=batch_size,
                                  weight_decay=0.01,
                                  evaluation_strategy="epoch",
                                  disable_tqdm=False,
                                  logging_steps=logging_steps,
                                  log_level="error",
                                  optim='adamw_torch'
                                  )
training_args

TrainingArguments(
_n_gpu=1,
adafactor=False,
adam_beta1=0.9,
adam_beta2=0.999,
adam_epsilon=1e-08,
auto_find_batch_size=False,
bf16=False,
bf16_full_eval=False,
data_seed=None,
dataloader_drop_last=False,
dataloader_num_workers=0,
dataloader_pin_memory=True,
ddp_bucket_cap_mb=None,
ddp_find_unused_parameters=None,
ddp_timeout=1800,
debug=[],
deepspeed=None,
disable_tqdm=False,
do_eval=True,
do_predict=False,
do_train=False,
eval_accumulation_steps=None,
eval_delay=0,
eval_steps=None,
evaluation_strategy=epoch,
fp16=False,
fp16_backend=auto,
fp16_full_eval=False,
fp16_opt_level=O1,
fsdp=[],
fsdp_min_num_params=0,
fsdp_transformer_layer_cls_to_wrap=None,
full_determinism=False,
gradient_accumulation_steps=1,
gradient_checkpointing=False,
greater_is_better=None,
group_by_length=False,
half_precision_backend=auto,
hub_model_id=None,
hub_private_repo=False,
hub_strategy=every_save,
hub_token=<HUB_TOKEN>,
ignore_data_skip=False,
include_inputs_for_metrics=False,
jit_mode_eval=False,
label_n

In [68]:
from transformers import Trainer

torch.cuda.empty_cache()

trainer = Trainer(model=model, 
                  args=training_args, 
                  train_dataset=tiny_imdb_encoded["train"],
                  eval_dataset=tiny_imdb_encoded["validation"],
                  tokenizer=tokenizer)
trainer.train();

Epoch,Training Loss,Validation Loss
1,0.695800,0.708748
2,0.668200,0.708855
3,0.669700,0.710295


In [69]:
preds = trainer.predict(tiny_imdb_encoded['test'])
preds

PredictionOutput(predictions=array([[ 0.20245673, -0.02454904],
       [ 0.20467982, -0.00127297],
       [ 0.21706797, -0.03786207],
       [ 0.22386877, -0.02881036],
       [ 0.21208312, -0.00520195],
       [ 0.20146877, -0.03909191],
       [ 0.22650993, -0.04083134],
       [ 0.20407619, -0.04807999],
       [ 0.2114564 , -0.05149288],
       [ 0.18646395, -0.01456954]], dtype=float32), label_ids=array([0, 0, 1, 1, 0, 0, 0, 0, 1, 0]), metrics={'test_loss': 0.6582468748092651, 'test_runtime': 0.0555, 'test_samples_per_second': 180.113, 'test_steps_per_second': 18.011})

In [70]:
preds.predictions.shape

(10, 2)

In [71]:
preds.predictions.argmax(axis=-1)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [72]:
preds.label_ids

array([0, 0, 1, 1, 0, 0, 0, 0, 1, 0])

In [73]:
from sklearn.metrics import accuracy_score

accuracy_score(preds.label_ids, preds.predictions.argmax(axis=-1))

0.7

In [74]:
def get_accuracy(preds):
  predictions = preds.predictions.argmax(axis=-1)
  labels = preds.label_ids
  accuracy = accuracy_score(preds.label_ids, preds.predictions.argmax(axis=-1))
  return {'accuracy': accuracy}


In [75]:
from transformers import Trainer

torch.cuda.empty_cache()

trainer = Trainer(model=model, 
                  compute_metrics=get_accuracy,
                  args=training_args, 
                  train_dataset=tiny_imdb_encoded["train"],
                  eval_dataset=tiny_imdb_encoded["validation"],
                  tokenizer=tokenizer)
trainer.train();

Epoch,Training Loss,Validation Loss,Accuracy
1,0.660900,0.698913,0.400000
2,0.630900,0.692877,0.400000
3,0.614900,0.692335,0.400000


## Training run

In [76]:
batch_size = 8
logging_steps = len(imdb_encoded["train"]) // batch_size
model_name = f"{checkpoint}-finetuned-imdb"
training_args = TrainingArguments(output_dir=model_name,
                                  num_train_epochs=2,
                                  learning_rate=2e-5,
                                  per_device_train_batch_size=batch_size,
                                  per_device_eval_batch_size=batch_size,
                                  weight_decay=0.01,
                                  evaluation_strategy="epoch",
                                  disable_tqdm=False,
                                  logging_steps=logging_steps,
                                  log_level="error",
                                  optim='adamw_torch'
                                  )

In [77]:
from transformers import Trainer

torch.cuda.empty_cache()

trainer = Trainer(model=model, 
                  args=training_args, 
                  compute_metrics=get_accuracy,
                  train_dataset=imdb_encoded["train"],
                  eval_dataset=imdb_encoded["validation"],
                  tokenizer=tokenizer)
trainer.train();

Epoch,Training Loss,Validation Loss,Accuracy
1,0.440500,0.454654,0.820000
2,0.214200,0.437962,0.860000


In [78]:


trainer.evaluate()

{'eval_loss': 0.4379621744155884,
 'eval_accuracy': 0.86,
 'eval_runtime': 1.8582,
 'eval_samples_per_second': 215.265,
 'eval_steps_per_second': 26.908,
 'epoch': 2.0}

In [79]:
trainer.save_model()

In [80]:
model_name

'distilbert-base-cased-finetuned-imdb'

In [81]:
from transformers import pipeline
classifier = pipeline('text-classification', model=model_name)
classifier('This is not my idea of fun')

[{'label': 'LABEL_0', 'score': 0.7977284789085388}]

In [82]:
classifier('Infinity Pool is definitely a wild time.')

[{'label': 'LABEL_1', 'score': 0.8837677836418152}]